# Parte C: Fuzzing basado en Búsqueda
En esta parte diseñaras un algoritmo de búsqueda para encontrar valores que cumplan con las condiciones dentro de una función. Comenzarás con una entrada aleatoria y, mediante una función objetivo, te acercaras progresivamente a la solución.

## Pre-requisitos

Antes de empezar, instala los siguientes pre-requisitos:

In [14]:
from IPython.display import clear_output
!apt-get update
!apt-get install -y graphviz graphviz-dev
!pip install pygraphviz
!pip install fuzzingbook
clear_output()

## Función a validar

## Implementación del algoritmo Search Based

In [15]:
def safety_check(temp, pressure, humidity):
    """
    Check if the machine's operational parameters are within a safe range.
    Returns True if the parameters are safe, otherwise False.
    """
    # Check temperature
    if temp >= 100:
        return False

    # Check pressure
    if pressure >= 200:
        return False

    # Check humidity
    if humidity < 20 or humidity > 70 :
        return False

    # Check combined condition for temperature and pressure
    if temp > 90 or pressure >= 180:
        return False

    # If all conditions are met, the parameters are safe
    return True

In [16]:
### Cotas valores de los vecinos
MAX = 1000
MIN = 0

### A) Defina la función neighbors:
Implementa la estrategia que prefiera para obtener los vecinos. Por ejemplo, sumando uno y restando uno a los valores recibidos de entrada.

Los valores de los vecinos no deben ser mayores a 1000, tampoco negativos, es decir que los valores deben variar entre 0 y 1000.

In [17]:
def neighbors(x, y, z):
    MAX = 1000
    MIN = 0
    step = 6 # tamaño del paso
    vecinas = []
    for dx in [-step, 0, step]:
        for dy in [-step, 0, step]:
            for dz in [-step, 0, step]:
                if dx == dy == dz == 0:
                    continue  # omitimos el punto original
                nx, ny, nz = x + dx, y + dy, z + dz
                if MIN <= nx <= MAX and MIN <= ny <= MAX and MIN <= nz <= MAX:
                    vecinas.append((nx, ny, nz))
    return vecinas

print(neighbors(10, 10, 10))


[(4, 4, 4), (4, 4, 10), (4, 4, 16), (4, 10, 4), (4, 10, 10), (4, 10, 16), (4, 16, 4), (4, 16, 10), (4, 16, 16), (10, 4, 4), (10, 4, 10), (10, 4, 16), (10, 10, 4), (10, 10, 16), (10, 16, 4), (10, 16, 10), (10, 16, 16), (16, 4, 4), (16, 4, 10), (16, 4, 16), (16, 10, 4), (16, 10, 10), (16, 10, 16), (16, 16, 4), (16, 16, 10), (16, 16, 16)]


### B) Defina una Función Fitness:
Usa el *branch-distance* como *fitness function*.

Recuerda que el branch-distance, es un valor que indica que tan cerca estan los valores de entrada en cumplir una condición.

Por ejemplo, considera esta condición `a > 100` con `a = 20`, esta condición deberia devolver `False`, y la distancia para que devuelva `True` es `81`, ya que si sumamos `80` a `20` tendriamos `101` y la condición seria `True`.

Por otro lado la distancia para que devuelva `True` es `0`, porque la condición ya evalua a `True`.

### B.1) Implemente la función `evaluate_condition`;
Cada vez que se evalúe una condición debe devolver el resultado de evaluar la condicion `(True, False)` de esta forma la función `instrumented_safety_check` funcionará igual que la original.

Además de devolver el valor, la función debe guardar el branch-distance de cada condición en una variable global (por ejemplo un diccionario).

Note que la distancia que nos interesa en este ejercicio es la distancia para condiciones evaluen `False`.

In [18]:
branch_distances = {}

def evaluate_condition(id, op, a, b):
    global branch_distances

    # Evaluar la condición
    if op == '>':
        result = a > b
        distance = max(0, b - a + 1)
    elif op == '>=':
        result = a >= b
        distance = max(0, b - a)
    elif op == '<':
        result = a < b
        distance = max(0, a - b + 1)
    elif op == '<=':
        result = a <= b
        distance = max(0, a - b)
    elif op == '==':
        result = a == b
        distance = 1 if a != b else 0
    else:
        result = False
        distance = 1000  # penalización

    # Guardar la distancia normalizada
    branch_distances[id] = distance / 1000 if not result else 0.0
    print(f"[CONDICIÓN {id}] {a} {op} {b} → resultado: {result}, distancia: {branch_distances[id]}")

    return result



In [19]:
def instrumented_safety_check(temp, pressure, humidity):
    """
    Check if the machine's operational parameters are within a safe range.
    Returns True if the parameters are safe, otherwise False.
    """
    # Check temperature
    if evaluate_condition(1, '>=',temp, 100):
        return False

    # Check pressure
    if evaluate_condition(2, '>=',pressure,200):
        return False

    # Check humidity
    if (evaluate_condition(3,'<',humidity,20) or evaluate_condition(4,'>',humidity, 70)) :
        return False

    # Check combined condition for temperature and pressure
    if evaluate_condition(5, '>',temp,90) or evaluate_condition(6,'>=',pressure,180):
        return False

    # If all conditions are met, the parameters are safe
    return True

### B.2) Implemente la fitness function;

Dado 3 valores devuelva el branch-distance total, que es la sumatoria de sumar las branch distance de todas las condiciones.

Para saber si fitness función esta bien, esta debe devolver `0` si los tres numeros ingresados como argumentos pasan todas las condiciones y la función `safety_check` devuelve verdad.

Si los valores estan cerca de cumplir las condiciones deben devolver un valor cercano a cero, si se aleja debe devolver un valor más grande.

Puedes probar esto, ejecutando la función con varios valores y ver si funciona. En la celda posterior a esta se ejemplifican dos casos.

In [20]:
def normalize(x):
    return x/ MAX

In [21]:
def get_fitness_validation(x, y, z):
    global branch_distances
    branch_distances = {}

    instrumented_safety_check(x, y, z)

    expected_ids = [1, 2, 3, 4, 5, 6]  # todas las condiciones esperadas
    fitness = 0.0

    for id in expected_ids:
        if id not in branch_distances:
            fitness += 1.0  # penalización por no haber sido evaluada
        else:
            fitness += branch_distances[id]

    return fitness



In [22]:
get_fitness_validation(894, 225, 24)



[CONDICIÓN 1] 894 >= 100 → resultado: True, distancia: 0.0


0.0

In [23]:
instrumented_safety_check(894, 225, 24)

[CONDICIÓN 1] 894 >= 100 → resultado: True, distancia: 0.0


False

## Probemos tu solución

In [24]:
import random

def hillclimb_validation(log=False):
    x,y,z = random.randint(MIN, MAX), random.randint(MIN, MAX),random.randint(MIN, MAX) # (1)
    fitness = get_fitness_validation(x,y,z) # (2)
    print("Initial input: %d %d %d at fitness %.4f" % (x,y,z, fitness))
    iterations = 0
    while fitness > 0: # (3)
        changed = False
        iterations += 1
        for (nextx,nexty,nextz) in neighbors(x,y,z): # (4)
            new_fitness = get_fitness_validation(nextx,nexty,nextz) # (5)
            if new_fitness < fitness:
                x,y,z = nextx,nexty,nextz
                fitness = new_fitness
                changed = True
                if log:
                  print("New value: %d, %d, %d at fitness %.4f" % (x, y, z, fitness))
                break

        if not changed: # (6)
            x, y, z = random.randint(MIN, MAX), random.randint(MIN, MAX),random.randint(MIN, MAX)
            fitness = get_fitness_validation(x,y,z)

    print("Found optimum after %d iterations at %d, %d, %d" % (iterations, x, y, z))
    return iterations

### Probando una iteración y mostrando logs (True)

*   Si tu solución es "adecuada" la fitness function deberia reducir en cada iteración, y terminar en `0` cuando encuentre la solución.



In [25]:
hillclimb_validation(True)

[CONDICIÓN 1] 694 >= 100 → resultado: True, distancia: 0.0
Initial input: 694 580 193 at fitness 0.0000
Found optimum after 0 iterations at 694, 580, 193


0

### Probando 100 iteraciones, sin logs

*   Esto es sobre todo para ver cuantas iteraciones toma en promedio, porque las iteraciones varian dependiendo el seed inicial que es aleatorio.

In [ ]:
population = []
for i in range(100):
  population.append(hillclimb_validation())
result = sum(population)
print("average")
result/100.0

Se truncaron las últimas líneas 5000 del resultado de transmisión.
[CONDICIÓN 1] 143 >= 100 → resultado: True, distancia: 0.0
[CONDICIÓN 1] 143 >= 100 → resultado: True, distancia: 0.0
[CONDICIÓN 1] 143 >= 100 → resultado: True, distancia: 0.0
[CONDICIÓN 1] 143 >= 100 → resultado: True, distancia: 0.0
[CONDICIÓN 1] 149 >= 100 → resultado: True, distancia: 0.0
[CONDICIÓN 1] 149 >= 100 → resultado: True, distancia: 0.0
[CONDICIÓN 1] 149 >= 100 → resultado: True, distancia: 0.0
[CONDICIÓN 1] 149 >= 100 → resultado: True, distancia: 0.0
[CONDICIÓN 1] 149 >= 100 → resultado: True, distancia: 0.0
[CONDICIÓN 1] 149 >= 100 → resultado: True, distancia: 0.0
[CONDICIÓN 1] 149 >= 100 → resultado: True, distancia: 0.0
[CONDICIÓN 1] 149 >= 100 → resultado: True, distancia: 0.0
[CONDICIÓN 1] 149 >= 100 → resultado: True, distancia: 0.0
[CONDICIÓN 1] 477 >= 100 → resultado: True, distancia: 0.0
[CONDICIÓN 1] 471 >= 100 → resultado: True, distancia: 0.0
[CONDICIÓN 1] 471 >= 100 → resultado: True, dist